In [57]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [58]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [59]:
daybreak = pd.read_csv(r"E:\Tasks\Pipeline\Daybreak Urban Core Phasing 2022_03_09 Bert.csv")
potm = pd.read_csv(r"E:\Tasks\Pipeline\POTM Pipeline Projects Phasing for REMM.xlsx - LHMDevZone_PipelineBuildingsCoded.csv")

In [60]:
daybreak.head()

,parcel_id_remm,year,parcel_acres,taz_id,taz_share,units,retail_jobspaces,retail_sqft,office_jobspaces,office_sqft,industrial_jobspaces,industrial_sqft,dev_type
0,27170,2040,2.230,1517,0.174,87,0,0,0,0,0,0,G
1,27170,2045,2.230,1517,0.174,87,0,0,422,105475,0,0,G
2,29630,2025,3.709,1520,0.108,41,0,0,2,381,0,0,G
3,29630,2030,3.709,1520,0.108,17,0,0,31,7850,0,0,G
4,29781,2040,1.000,1522,0.053,9,0,0,0,0,0,0,G


In [61]:
daybreak.rename({'parcel_id_remm':'parcel_id'}, axis=1, inplace=True)
daybreak.rename({'year':'year_built'}, axis=1, inplace=True)
daybreak.rename({'units':'residential_units'}, axis=1, inplace=True)
daybreak['job_spaces'] =  daybreak['retail_jobspaces'] + daybreak['office_jobspaces'] + daybreak['industrial_jobspaces']
daybreak['non_residential_sqft'] =  daybreak['retail_sqft'] + daybreak['office_sqft'] + daybreak['industrial_sqft']
daybreak['building_sqft'] =  daybreak['retail_sqft'] + daybreak['office_sqft'] + daybreak['industrial_sqft']

# mixed use
daybreak.loc[(daybreak['residential_units'] > 0) & ( (daybreak['industrial_jobspaces'] > 0)|(daybreak['retail_jobspaces'] > 0)|(daybreak['office_jobspaces'] > 0)) ,'building_type_id'] = 7
# residential
daybreak.loc[(daybreak['residential_units'] > 0) & ( (daybreak['industrial_jobspaces'] == 0) & (daybreak['retail_jobspaces'] == 0) & (daybreak['office_jobspaces'] == 0)) ,'building_type_id'] = 2
# industrial
daybreak.loc[(daybreak['residential_units'] == 0) & ( (daybreak['industrial_sqft'] > daybreak['office_sqft']) | (daybreak['industrial_sqft'] > daybreak['retail_sqft'])) ,'building_type_id'] = 3
# office
daybreak.loc[(daybreak['residential_units'] == 0) & ( (daybreak['office_sqft'] > daybreak['retail_sqft']) | (daybreak['office_sqft'] > daybreak['industrial_sqft'])) ,'building_type_id'] = 5
# retail
daybreak.loc[(daybreak['residential_units'] == 0) & ( (daybreak['retail_sqft'] > daybreak['office_sqft']) | (daybreak['retail_sqft'] > daybreak['industrial_sqft'])) ,'building_type_id'] = 5

daybreak['note'] = 'pipeline'
daybreak['DEVTYPE'] = 'develop'
daybreak['stories'] = 1
daybreak['non_residential_sqft'] = 0
daybreak['unit_price_non_residential'] = 0
daybreak['res_price_per_sqft'] = 0
daybreak['residential_price'] = 0
daybreak['non_residential_price'] = 0
daybreak['source'] = 'daybreak urban core'

daybreak = daybreak[['building_type_id','non_residential_sqft','note','parcel_id','residential_units','stories','unit_price_non_residential',
                     'year_built','res_price_per_sqft','residential_price','non_residential_price','job_spaces','building_sqft','DEVTYPE','source']]

In [62]:
daybreak

,building_type_id,non_residential_sqft,note,parcel_id,residential_units,stories,unit_price_non_residential,year_built,res_price_per_sqft,residential_price,non_residential_price,job_spaces,building_sqft,DEVTYPE,source
0,2.0,0,pipeline,27170,87,1,0,2040,0,0,0,0,0,develop,daybreak urban core
1,7.0,0,pipeline,27170,87,1,0,2045,0,0,0,422,105475,develop,daybreak urban core
2,7.0,0,pipeline,29630,41,1,0,2025,0,0,0,2,381,develop,daybreak urban core
3,7.0,0,pipeline,29630,17,1,0,2030,0,0,0,31,7850,develop,daybreak urban core
4,2.0,0,pipeline,29781,9,1,0,2040,0,0,0,0,0,develop,daybreak urban core
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,5.0,0,pipeline,756362,0,1,0,2035,0,0,0,282,70547,develop,daybreak urban core
346,5.0,0,pipeline,756633,0,1,0,2025,0,0,0,108,77392,develop,daybreak urban core
347,7.0,0,pipeline,756633,50,1,0,2030,0,0,0,61,24667,develop,daybreak urban core
348,5.0,0,pipeline,756634,0,1,0,2025,0,0,0,108,77393,develop,daybreak urban core


In [63]:
daybreak.to_csv(os.path.join(outputs[0], 'daybreak_urban_core_pipeline.csv'),index=False)

In [64]:
potm.head()

,parcel_id_remm,year,units,retail_jobspaces,retail_sqft,office_jobspaces,office_sqft,industrial_jobspaces,industrial_sqft,dev_type,notes
0,35183,2025,370,54,21600,1000,300000,0,0,G,NaN
1,35183,2026,370,54,21600,1000,300000,0,0,G,NaN
2,35183,2027,370,54,21600,1000,300000,0,0,G,NaN
3,35183,2028,370,54,21600,1000,300000,0,0,G,NaN
4,35183,2029,370,54,21600,1000,300000,0,0,G,NaN


In [65]:
potm.rename({'parcel_id_remm':'parcel_id'}, axis=1, inplace=True)
potm.rename({'year':'year_built'}, axis=1, inplace=True)
potm.rename({'units':'residential_units'}, axis=1, inplace=True)
potm['job_spaces'] =  potm['retail_jobspaces'] + potm['office_jobspaces'] + potm['industrial_jobspaces']
potm['non_residential_sqft'] =  potm['retail_sqft'] + potm['office_sqft'] + potm['industrial_sqft']
potm['building_sqft'] =  potm['retail_sqft'] + potm['office_sqft'] + potm['industrial_sqft']

# mixed use
potm.loc[(potm['residential_units'] > 0) & ( (potm['industrial_jobspaces'] > 0)|(potm['retail_jobspaces'] > 0)|(potm['office_jobspaces'] > 0)) ,'building_type_id'] = 7
# residential
potm.loc[(potm['residential_units'] > 0) & ((potm['industrial_jobspaces'] == 0) & (potm['retail_jobspaces'] == 0) & (potm['office_jobspaces'] == 0)) ,'building_type_id'] = 2
# industrial
potm.loc[(potm['residential_units'] == 0) & ( (potm['industrial_sqft'] > potm['office_sqft']) | (potm['industrial_sqft'] > potm['retail_sqft'])) ,'building_type_id'] = 3
# office
potm.loc[(potm['residential_units'] == 0) & ( (potm['office_sqft'] > potm['retail_sqft']) | (potm['office_sqft'] > potm['industrial_sqft'])) ,'building_type_id'] = 5
# retail
potm.loc[(potm['residential_units'] == 0) & ( (potm['retail_sqft'] > potm['office_sqft']) | (potm['retail_sqft'] > potm['industrial_sqft'])) ,'building_type_id'] = 5

potm['note'] = 'pipeline'
potm['DEVTYPE'] = 'develop'
potm['stories'] = 1
potm['non_residential_sqft'] = 0
potm['unit_price_non_residential'] = 0
potm['res_price_per_sqft'] = 0
potm['residential_price'] = 0
potm['non_residential_price'] = 0
potm['source'] = 'point of the mountain'

potm = potm[['building_type_id','non_residential_sqft','note','parcel_id','residential_units','stories','unit_price_non_residential',
                     'year_built','res_price_per_sqft','residential_price','non_residential_price','job_spaces','building_sqft','DEVTYPE', 'source']]

In [66]:
potm.to_csv(os.path.join(outputs[0], 'point_of_the_mountain_pipeline.csv'), index=False)

# Building Permit Data

In [67]:
parcels = r'E:\Tasks\Replace_HAFB_Parcels\Outputs\remm_base_year_20220513.gdb\parcels_2019'
cm = r"E:\REMM\Pipeline_Projects\Data\REMM Pipeline Projects.gdb\ALL_ConstructionMonitor_Data_2014_2020"
cm_layer = arcpy.MakeFeatureLayer_management(cm, 'cm_lyr')

In [68]:
# Demolition Residential
query = """const_type IN ('Demolition') And pmt_class = 'Residential' And pmt_beg > timestamp '2019-12-01 11:24:08'"""
arcpy.SelectLayerByAttribute_management(cm_layer, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(cm_layer, 'INTERSECT', parcels, selection_type='SUBSET_SELECTION' )

target_features = cm_layer
join_features = parcels
output_features = os.path.join(gdb, "demo_pts_residential")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# run the spatial join
sj1 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(cm_layer, 'CLEAR_SELECTION', query)

<Result 'cm_lyr'>

In [69]:
d_res = pd.DataFrame.spatial.from_featureclass(sj1[0])
d_res = d_res[d_res['year_built'] > 0].copy()
d_res['age'] = 2020 - d_res['year_built']
d_res = d_res[d_res['age'] >= 50].copy() # REMM redev age for residential
d_res.loc[d_res['pmt_beg'].dt.month <= 6, 'year_demo'] =d_res['pmt_beg'].dt.year
d_res.loc[d_res['pmt_beg'].dt.month > 6, 'year_demo'] =d_res['pmt_beg'].dt.year +1 

# sort on date then keep the most recent duplicate
d_res.sort_values('year_demo', inplace=True)
d_res.drop_duplicates(subset = ['site_lot'], keep='last', inplace=True)

d_res = d_res[['parcel_id_REMM','year_demo']].copy()
d_res.rename({'parcel_id_REMM':'parcel_id', 'year_demo':'year_built'}, axis=1, inplace=True)
d_res['building_type_id'] = 0
d_res['non_residential_sqft'] = 0
d_res['residential_units'] = 0
d_res['unit_price_non_residential'] = 0
d_res['non_residential_price'] = 0
d_res['job_spaces'] = 0
d_res['building_sqft'] = 0
d_res['DEVTYPE'] = 'demolition'
d_res['source'] = 'construction monitor - demolition residential'
d_res.drop_duplicates(inplace=True)
d_res['parcel_id'].value_counts()

449505    1
132387    1
117940    1
88921     1
260151    1
453783    1
390985    1
55275     1
55267     1
498276    1
42798     1
159695    1
Name: parcel_id, dtype: int64

In [70]:
d_res.to_csv(os.path.join(outputs[0], 'demolition_residential.csv'),index=False)

In [71]:
# Demolition Commercial
query = """const_type IN ('Demolition','Demolition (Commercial)') And pmt_class <> 'Residential'  And pmt_beg > timestamp '2019-12-01 11:24:08'"""
arcpy.SelectLayerByAttribute_management(cm_layer, 'NEW_SELECTION', query)
arcpy.SelectLayerByLocation_management(cm_layer, 'INTERSECT', parcels, selection_type='SUBSET_SELECTION' )

target_features = cm_layer
join_features = parcels
output_features = os.path.join(gdb, "demo_pts_commercial")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# run the spatial join
sj2 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(cm_layer, 'CLEAR_SELECTION', query)

<Result 'cm_lyr'>

In [72]:
d_nonres = pd.DataFrame.spatial.from_featureclass(sj2[0])
d_nonres = d_nonres[d_nonres['year_built'] > 0].copy()
d_nonres['age'] = 2020 - d_nonres['year_built']
d_nonres = d_nonres[d_nonres['age'] >= 20].copy() # REMM redev age for residential
d_nonres.loc[d_nonres['pmt_beg'].dt.month <= 6, 'year_demo'] =d_nonres['pmt_beg'].dt.year
d_nonres.loc[d_nonres['pmt_beg'].dt.month > 6, 'year_demo'] =d_nonres['pmt_beg'].dt.year +1 

# sort on date then keep the most recent duplicate
d_nonres.sort_values('year_demo', inplace=True)
d_nonres.drop_duplicates(subset = ['site_lot'], keep='last', inplace=True)

d_nonres = d_nonres[['parcel_id_REMM','year_demo']].copy()
d_nonres.rename({'parcel_id_REMM':'parcel_id', 'year_demo':'year_built'}, axis=1, inplace=True)
d_nonres['building_type_id'] = 0
d_nonres['non_residential_sqft'] = 0
d_nonres['residential_units'] = 0
d_nonres['unit_price_non_residential'] = 0
d_nonres['non_residential_price'] = 0
d_nonres['job_spaces'] = 0
d_nonres['building_sqft'] = 0
d_nonres['DEVTYPE'] = 'demolition'
d_nonres['source'] = 'construction monitor - demolition non-residential'
d_nonres.drop_duplicates(inplace=True)
d_nonres['parcel_id'].value_counts()

36032     1
129313    1
50114     1
2626      1
72085     1
72086     1
38592     1
121147    1
55279     1
Name: parcel_id, dtype: int64

In [73]:
d_nonres.to_csv(os.path.join(outputs[0], 'demolition_non_residential.csv'),index=False)

In [74]:
# single family vacant or agriculture land
query = """ const_type IN ('Single Family Homes') And pmt_beg > timestamp '2019-12-01 11:24:08' """
arcpy.SelectLayerByAttribute_management(cm_layer, 'NEW_SELECTION', query)

query2 = """ building_type IN ('Empty Buildable', 'Agriculture') """
parcels_lyr = arcpy.SelectLayerByAttribute_management(parcels, 'NEW_SELECTION', query2)
arcpy.SelectLayerByLocation_management(cm_layer, 'INTERSECT', parcels_lyr, selection_type='SUBSET_SELECTION' )

target_features = cm_layer
join_features = parcels
output_features = os.path.join(gdb, "single_family_vacant")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# run the spatial join
sj3 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_COMMON", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(cm_layer, 'CLEAR_SELECTION', query)
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION', query)

<Result 'parcels_2019_Layer14'>

In [75]:
b_single = pd.DataFrame.spatial.from_featureclass(sj3[0])
b_single['year_build'] = b_single['pmt_beg'].dt.year +1

# sort on date then keep the most recent duplicate
b_single.sort_values('year_build', inplace=True)
b_single.drop_duplicates(subset = ['site_lot'], keep='last', inplace=True)

b_single = b_single[['parcel_id_REMM','year_build','pmt_sqft', 'pmt_units']].copy()
b_single.rename({'parcel_id_REMM':'parcel_id', 'year_build':'year_built','pmt_units':'residential_units','pmt_sqft':'building_sqft'}, axis=1, inplace=True)
b_single['building_type_id'] = 1
b_single['non_residential_sqft'] = 0
b_single['unit_price_non_residential'] = 0
b_single['non_residential_price'] = 0
b_single['job_spaces'] = 0
b_single['DEVTYPE'] = 'develop'
b_single['source'] = 'construction monitor single family vacant'

In [76]:
sums = b_single.groupby('parcel_id')[['building_sqft', 'residential_units']].sum().reset_index().sort_values('residential_units')
del b_single['building_sqft']
del b_single['residential_units']
b_single= b_single.merge(sums, on='parcel_id', how='left')
b_single.loc[b_single['residential_units'] == 0, 'residential_units'] = 1
b_single.drop_duplicates(subset = ['parcel_id','residential_units'], inplace=True)
b_single['parcel_id'].value_counts()

499712    1
578881    1
508356    1
509260    1
69269     1
         ..
502381    1
508522    1
500328    1
477800    1
499710    1
Name: parcel_id, Length: 1230, dtype: int64

In [77]:
b_single.to_csv(os.path.join(outputs[0], 'develop_single_family_vacant.csv'),index=False)

In [78]:
# multi family
query = """ const_type IN ('Apartments & Condos') And pmt_beg > timestamp '2019-12-01 11:24:08' """
arcpy.SelectLayerByAttribute_management(cm_layer, 'NEW_SELECTION', query)

query2 = """ building_type IN ('Empty Buildable', 'Agriculture') And parcel_acres > 0.05 """
parcels_lyr = arcpy.SelectLayerByAttribute_management(parcels, 'NEW_SELECTION', query2)
arcpy.SelectLayerByLocation_management(cm_layer, 'INTERSECT', parcels_lyr, selection_type='SUBSET_SELECTION' )

target_features = cm_layer
join_features = parcels
output_features = os.path.join(gdb, "multi_family_vacant")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# run the spatial join
sj4 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_COMMON", 
                           fieldmappings, "INTERSECT")

arcpy.SelectLayerByAttribute_management(cm_layer, 'CLEAR_SELECTION', query)
arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION', query)

<Result 'parcels_2019_Layer17'>

In [79]:
b_multi = pd.DataFrame.spatial.from_featureclass(sj3[0])
b_multi = b_multi[b_multi['pmt_units'] > 0].copy()
b_multi['year_build'] = b_multi['pmt_beg'].dt.year +1

# sort on date then keep the most recent duplicate
b_multi.sort_values('year_build', inplace=True)
b_multi.drop_duplicates(subset = ['site_lot'], keep='last', inplace=True)

b_multi = b_multi[['parcel_id_REMM','year_build','pmt_sqft', 'pmt_units']].copy()
b_multi.rename({'parcel_id_REMM':'parcel_id', 'year_build':'year_built','pmt_units':'residential_units','pmt_sqft':'building_sqft'}, axis=1, inplace=True)
b_multi['building_type_id'] = 2
b_multi['non_residential_sqft'] = 0
b_multi['unit_price_non_residential'] = 0
b_multi['non_residential_price'] = 0
b_multi['job_spaces'] = 0
b_multi['DEVTYPE'] = 'develop'
b_multi['source'] = 'construction monitor multifamily vacant'

In [80]:
sums = b_multi.groupby('parcel_id')[['building_sqft', 'residential_units']].sum().reset_index().sort_values('residential_units')
del b_multi['building_sqft']
del b_multi['residential_units']
b_multi.drop_duplicates(inplace=True)
b_multi = b_multi.merge(sums, on='parcel_id', how='left')
b_multi['parcel_id'].value_counts()

665389    2
499712    1
502258    1
576841    1
542023    1
         ..
502381    1
508522    1
500328    1
477800    1
499710    1
Name: parcel_id, Length: 1230, dtype: int64

In [81]:
b_multi.to_csv(os.path.join(outputs[0], 'develop_multifamily_vacant.csv'),index=False)

In [82]:
# not enough to explore...

# # non res vacant

# query = """const_type NOT IN ('Apartments & Condos', 'Demolition', 'Demolition (Commercial)', 'Single Family Homes','Comm Rmdl, Addn, Int Fin', 'Reroof Commercial', 'Reroof Residential', 'Res Rmdl, Addn, Int Fin', 'Swimming Pools & Spas', 
#             'Utilities (gas elect wtr swr)', 'Parking Structures & Carports', 'Garages & Carports') 
#             And pmt_beg >= timestamp '2019-12-01 11:24:08' 
#             And pmt_descrp NOT LIKE '%"Accessory"%' And pmt_sqft > 0"""

# arcpy.SelectLayerByAttribute_management(cm_layer, 'NEW_SELECTION', query)

# query2 = """ building_type IN ('Empty Buildable', 'Agriculture') And parcel_acres > 0.05 """
# parcels_lyr = arcpy.SelectLayerByAttribute_management(parcels, 'NEW_SELECTION', query2)
# arcpy.SelectLayerByLocation_management(cm_layer, 'INTERSECT', parcels_lyr, selection_type='SUBSET_SELECTION' )

# target_features = cm_layer
# join_features = parcels
# output_features = os.path.join(gdb, "multi_family_vacant")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # run the spatial join
# sj5 = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_COMMON", 
#                            fieldmappings, "INTERSECT")

# arcpy.SelectLayerByAttribute_management(cm_layer, 'CLEAR_SELECTION', query)
# arcpy.SelectLayerByAttribute_management(parcels_lyr, 'CLEAR_SELECTION', query)

In [83]:
# query = """const_type NOT IN ('Comm Rmdl, Addn, Int Fin', 'Reroof Commercial', 'Reroof Residential', 'Res Rmdl, Addn, Int Fin', 'Swimming Pools & Spas', 'Utilities (gas elect wtr swr)', 'Parking Structures & Carports', 'Garages & Carports') 
#             And pmt_beg >= timestamp '2020-01-03 00:00:00' 
#             And pmt_descrp NOT LIKE '%"Accessory"%' And pmt_sqft > 0"""

# arcpy.SelectLayerByAttribute_management(cm_layer, 'NEW_SELECTION', query)
# arcpy.SelectLayerByLocation_management(cm_layer, 'INTERSECT', parcels, selection_type='SUBSET_SELECTION' )

# useable = arcpy.FeatureClassToFeatureClass_conversion(cm_layer, gdb, 'permit_selection')

In [84]:
# query = """building_type = 'Empty Buildable'"""
# vacant_parcels = arcpy.SelectLayerByAttribute_management(parcels, 'NEW_SELECTION', query)

# new_builds = arcpy.SelectLayerByLocation_management(useable, 'INTERSECT', vacant_parcels, selection_type='NEW_SELECTION' )
# arcpy.FeatureClassToFeatureClass_conversion(new_builds, gdb, 'new_builds')